#**Prep**

##*installing needed libraries*

In [ ]:
!pip install osmnx
!pip install momepy
!pip install --upgrade xlrd

In [ ]:
import osmnx as ox
import momepy
import matplotlib
import networkx as nx
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import shapely
import geopandas as gpd
import itertools

In [ ]:
%cd /content/drive/MyDrive/THESIS - SNG

/content/drive/MyDrive/THESIS - SNG


In [ ]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.2.1

     |████████████████████████████████| 435 kB 5.1 MB/s 
     |████████████████████████████████| 500 kB 63.1 MB/s 
     |████████████████████████████████| 120 kB 73.1 MB/s 
     |████████████████████████████████| 1.6 MB 61.4 MB/s 
     |████████████████████████████████| 82 kB 678 kB/s 
     |████████████████████████████████| 41 kB 740 kB/s 


In [ ]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.2.1")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.2.1, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [ ]:
from stellargraph import StellarGraph
import math
from statistics import mean

##*getting locations for street network retrieval from osmnx*

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('final dataset_locations').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
geocodes_dataframe = pd.DataFrame.from_records(rows)
geocodes_dataframe, geocodes_dataframe.columns = geocodes_dataframe[1:] , geocodes_dataframe.iloc[0]

[['Geocode', 'Location', 'Typology'], ['Carrer de Martínez de la Rosa, la Vila de Gràcia, Gràcia, Barcelona, Barcelonès, Barcelona, Catalonia, 08001, Spain', '41.39935, 2.15934', '2'], ['Carrer de Galileu, Sants, Sants-Montjuïc, Barcelona, Barcelonès, Barcelona, Catalonia, 08001, Spain', '41.37804, 2.13498', '2'], ['Carrer de Sant Carles, la Barceloneta, Ciutat Vella, Barcelona, Barcelonès, Barcelona, Catalonia, 08001, Spain', '41.37924, 2.18961', '1'], ['Carrer de Rossell, la Teixonera, Horta-Guinardó, Barcelona, Barcelonès, Barcelona, Catalonia, 08001, Spain', '41.42258, 2.14568', '3'], ['Carrer de Lleida, Cella Vinaria, Teià, Maresme, Barcelona, Catalonia, 08329, Spain', '41.4913, 2.3142', '4'], ["Avinguda d'Antoni Gaudí, Alella Park, Alella, Maresme, Barcelona, Catalonia, 08328, Spain", '41.51231, 2.29035', '3'], ['ronda del Pirineu, Can Soberano, Premià de Dalt, Maresme, Barcelona, Catalonia, 08338, Spain', '41.49868, 2.34009', '4'], ['Carrer del Montseny, Vilassar de Dalt, Maresm

In [ ]:
geocodes_dataframe

,Geocode,Location,Typology
1,"Carrer de Martínez de la Rosa, la Vila de Gràc...","41.39935, 2.15934",2
2,"Carrer de Galileu, Sants, Sants-Montjuïc, Barc...","41.37804, 2.13498",2
3,"Carrer de Sant Carles, la Barceloneta, Ciutat ...","41.37924, 2.18961",1
4,"Carrer de Rossell, la Teixonera, Horta-Guinard...","41.42258, 2.14568",3
5,"Carrer de Lleida, Cella Vinaria, Teià, Maresme...","41.4913, 2.3142",4
...,...,...,...
402,"BMS, أبو بيكر بن عامر, Nouakchott, Ksar, Nouak...","18.10366, -15.95562",1
403,"Etweymiret, Kiffa, Assaba, Mauritania","16.61953, -11.39744",1
404,"Boulevard Mohamed V شارع محمد الخامس, Nador, P...","35.18046, -2.92422",1
405,"Hay karrakchou, Hay Al Majd, Berkane, Pachalik...","34.94159, -2.33155",3


In [ ]:
cities =[i for i in geocodes_dataframe.Location] 

cities =[tuple(map(float, i.split(', '))) for i in cities] 
type(cities)

list

## *Saving images of the retrieved networks*

In [ ]:
index = 75

for city in cities[75:]:
  G = ox.graph_from_point(city, dist=300, network_type='drive', simplify=False)
  ox.plot_graph(G, ax = None, figsize=(5,5),bgcolor='white', show=True,node_size=0, edge_linewidth=2, edge_color="#000000", save=True, 
                filepath="/content/drive/MyDrive/THESIS - SNG/01. DATASET/final dataset/plots/{}.png".format(index), 
                dpi = 100)
  index+=1

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

def load_images_from_folder(folder):
    images = []
    filenames = glob.glob(os.path.join(folder,"*"))
    for i in range(0,len(filenames)):
      img = cv2.imread(os.path.join(folder,'{}.png'.format(i)))
      if img is not None:
        #img = cv2.resize(img, (150, 150))   
        images.append(img)
    return images

#**Dataset creation**

##*- primal approach +/- features*

In [ ]:
#no features

#instantiate empty list to which Stellar graph instances will be appended
graphs =[]
graphnr =1

for city in cities:
  G = ox.graph_from_point(city, dist=300, network_type='drive', simplify=True)
  G = ox.utils_graph.get_undirected(G)

  G = ox.project_graph(G)
  nodes, edges = ox.utils_graph.graph_to_gdfs(G)


  nodes = pd.DataFrame().assign(street_count=nodes['street_count'])
  nodes=nodes.replace(' ', 0) 

  edges = edges.reset_index()
  edges = pd.DataFrame().assign(source=edges['from'], target=edges['to'], weight=edges['length'])
  edges=edges.replace(' ', 0) 

  citygraph = StellarGraph(nodes, edges)
  graphs.append(citygraph)

  print('graph number                    == {}'.format(graphnr))
  graphnr+=1

In [ ]:
#with features

#instantiate empty list to which Stellar graph instances will be appended
graphs =[]
graphnr =1

for city in cities:
  G = ox.graph_from_point(city, dist=300, network_type='drive', simplify=True)
  G = ox.utils_graph.get_undirected(G)

  ox.bearing.add_edge_bearings(G, precision=1)
  G = ox.project_graph(G)
  nodes, edges = ox.utils_graph.graph_to_gdfs(G)

  nodes["avg_angle"] = " "
  nodes["max_angle"] = " "
  nodes["min_angle"] = " "
  nodes["prop_angle"] = " "
  nodes["prop_length"] = " "

  for index in nodes.index:
  # nodes with street count <2 are dead ends, we dont give attributes to them
    if nodes.at[index,'street_count']>=2:
      index = int(index)
      #get the lengths of the edges the node is either a startpoint or an endpoint
      dl = edges.query('u == {}'.format(index)).length
      dl=dl.append(edges.query('v == {}'.format(index)).length)
      #convert the created dataframe to a list
      length = dl.tolist()
      #assign the feature to the respective node column
      if min(length)!=0:
        nodes.at[index,'prop_length'] = round(max(length)/min(length),2)
      #get the calculated bearing of the edges the node is either a startpoint or an endpoint
      
      #for calculating the angles between intersecting streets we use the calculated bearing in osmnx
      #bearing is the angle between the geodesic north and the edge
      df1 = edges.query('u == {}'.format(index)).bearing
      df2 = edges.query('v == {}'.format(index)).bearing
      
      #angle is max-min angle of the 2 edges, note that when the node is v of the edge we need to add 180 degrees to the bearing
      #thats why we divide them in 2 lists
      ang_u= df1.tolist()

      ang_v= df2.tolist()
      ang_v =[x+180 for x in ang_v]

      angles = ang_u+ang_v

      if len(angles)>1:
        #create pairs of the streets in the intersection
        street_pairs = list(itertools.combinations(angles, 2))
        int_angles =[]
        for streets in street_pairs:
          int_angles.append(max(streets)-min(streets))
        int_angles = [x for x in int_angles if math.isnan(x) == False]

        if len(int_angles)>0:
          nodes.at[index,'avg_angle'] = int(mean(int_angles))
          nodes.at[index,'max_angle'] = int(max(int_angles))
          nodes.at[index,'min_angle'] = int(min(int_angles))
          if int(min(int_angles))==0:
            continue
          else:
            nodes.at[index,'prop_angle'] = round(int(max(int_angles))/int(min(int_angles)),2)

  nodes = pd.DataFrame().assign(street_count=nodes['street_count'], avg_angle=nodes['avg_angle'], max_angle=nodes['max_angle'],min_angle=nodes['min_angle'],prop_angle=nodes['prop_angle'],prop_length=nodes['prop_length'])
  nodes=nodes.replace(' ', 0) 

  edges = edges.reset_index()
  edges = pd.DataFrame().assign(source=edges['from'], target=edges['to'], weight=edges['length'])
  edges=edges.replace(' ', 0) 

  citygraph = StellarGraph(nodes, edges)
  graphs.append(citygraph)

  print('graph number                    == {}'.format(graphnr))
  graphnr+=1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#save to the desired location
import pickle
# Dump graph
with open("/content/drive/MyDrive/THESIS - SNG/01. DATASET/final dataset/stellargraphs_final_nofeatures.p", 'wb') as f:
    pickle.dump(graphs, f)

##*- dual approach +/- features*

In [ ]:
#with features
#instantiate empty list to which Stellar graph instances will be appended
graphs =[]
graphnr =1

for city in cities:
  G = ox.graph_from_point(city, dist=300, network_type='drive', simplify=True)
  G = ox.utils_graph.get_undirected(G)

  ox.bearing.add_edge_bearings(G, precision=1)
  G = ox.project_graph(G)
  nodes, edges = ox.utils_graph.graph_to_gdfs(G)

  edges['length'] = edges['length'].fillna(1)
  edges['bearing'] = edges['bearing'].fillna(1)

  dual = momepy.gdf_to_nx(edges, approach='dual')
  for node_id, node_data in dual.nodes(data=True):
      node_data["features"] = [round(dual.nodes[node_id]['length']), round(dual.nodes[node_id]['bearing'])]

  sgraph = StellarGraph.from_networkx(dual, node_features="features")
  graphs.append(sgraph)

  print('graph number                    == {}'.format(graphnr))
  graphnr+=1
  

In [ ]:
#without features
#instantiate empty list to which Stellar graph instances will be appended
graphs =[]
graphnr =1

for city in cities:
  G = ox.graph_from_point(city, dist=300, network_type='drive', simplify=True)
  G = ox.utils_graph.get_undirected(G)

  nodes, edges = ox.utils_graph.graph_to_gdfs(G)
  G = ox.project_graph(G)
  dual = momepy.gdf_to_nx(edges, approach='dual')
  for node_id, node_data in dual.nodes(data=True):
      node_data["features"] = [1]

  sgraph = StellarGraph.from_networkx(dual, node_features="features")
  graphs.append(sgraph)

  print('graph number                    == {}'.format(graphnr))
  graphnr+=1
  

In [ ]:
#save to the desired location
import pickle
# Dump graph
with open("/content/drive/MyDrive/THESIS - SNG/01. DATASET/final dataset/dual_final.p", 'wb') as f:
    pickle.dump(graphs, f)

##*Load dataset - check*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
# Load graph
with open("path/stellargraphs_final.p", 'rb') as f:  # notice the r instead of w
    sgraphs = pickle.load(f)

In [ ]:
print(sgraphs[1].info())

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in sgraphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

##*Export files for street decoder*

In [ ]:

G = ox.graph_from_point(cities[400], dist=300, network_type='drive', simplify=False)
G = ox.utils_graph.get_undirected(G)

ox.bearing.add_edge_bearings(G, precision=1)
ox.projection.project_graph(G, to_crs="epsg:3345")

nodes, edges = ox.utils_graph.graph_to_gdfs(G)

edges = edges.reset_index()
edges = pd.DataFrame().assign(source=edges['from'], target=edges['to'], length=edges['length'], bearing = edges['bearing'])
edges=edges.replace(' ', 0) 

index = nodes.index.tolist()
np.savetxt(r'/content/drive/MyDrive/THESIS - SNG/01. DATASET/encoding street networks files/bearing/400/nodeIndex.txt', index, delimiter= ",", fmt='%f')
np.savetxt(r'/content/drive/MyDrive/THESIS - SNG/01. DATASET/encoding street networks files/bearing/400/edges.txt', edges.values, delimiter= ",", fmt='%f' )